In [19]:
import tensorflow as tf

In [20]:
model = tf.keras.models.load_model("opennmt")


In [21]:
import pyonmttok
import os

In [22]:
export_dir = "opennmt"

In [23]:
imported = tf.saved_model.load(export_dir)
translate_fn = imported.signatures["serving_default"]
sp_model_path = os.path.join(export_dir, "assets.extra", "wmtende.model")
tokenizer = pyonmttok.Tokenizer("none", sp_model_path=sp_model_path)

In [45]:
def preprocess(texts):
        all_tokens = []
        lengths = []
        max_length = 0
        for text in texts: 
            tokens, _ = tokenizer.tokenize(text)
            length = len(tokens)
            all_tokens.append(tokens)
            lengths.append(length)
            max_length = max(max_length, length) 
        
        for tokens, length in zip(all_tokens, lengths):
            if length < max_length:
                tokens += [""] * (max_length - length)

        inputs = {
            "tokens": tf.constant(all_tokens, dtype=tf.string),
            "length": tf.constant(lengths, dtype=tf.int32),
        }
        return 

In [46]:
print(preprocess(['how are you?']))

how are you? ['▁how', '▁are', '▁you', '?'] 4 4
None


In [25]:
def postprocess(outputs):
    texts = []
    for tokens, length in zip(outputs["tokens"].numpy(), outputs["length"].numpy()):
        tokens = tokens[0][: length[0]].tolist()
        texts.append(tokenizer.detokenize(tokens))
    return texts

In [28]:
def translate(texts):
        """Translates a batch of texts."""
        inputs = preprocess(texts)
        print(inputs)
        #outputs = translate_fn(**inputs)
        #return postprocess(outputs)

how are you?
None


In [34]:
text = input("Source: ")
text

'how are you?'

'ola'